In [1]:
from strands import Agent
from strands.models.ollama import OllamaModel
import pprint 

In [2]:
def custom_callback_handler(**kwargs):
    # Access request state
    if "request_state" in kwargs:
        state = kwargs["request_state"]
        if "counter" not in state:
            state["counter"] = 0
        state["counter"] += 1
        print(f"Callback handler event count: {state['counter']}")

In [3]:
model = OllamaModel(
    host="http://localhost:11434",
    model_id="llama3.1",
)

agent = Agent(
    model=model,
    callback_handler=custom_callback_handler
)

In [4]:
result = agent("Hi there!")

Callback handler event count: 1
Callback handler event count: 2
Callback handler event count: 3
Callback handler event count: 4
Callback handler event count: 5
Callback handler event count: 6
Callback handler event count: 7
Callback handler event count: 8
Callback handler event count: 9
Callback handler event count: 10
Callback handler event count: 11
Callback handler event count: 12
Callback handler event count: 13
Callback handler event count: 14
Callback handler event count: 15
Callback handler event count: 16
Callback handler event count: 17
Callback handler event count: 18
Callback handler event count: 19
Callback handler event count: 20
Callback handler event count: 21


In [5]:
pprint.pprint(result)

AgentResult(stop_reason='end_turn',
            message={'content': [{'text': "How's it going? Is there something "
                                          'I can help you with or would you '
                                          'like to chat?'}],
                     'role': 'assistant'},
            metrics=EventLoopMetrics(cycle_count=1,
                                     tool_metrics={},
                                     cycle_durations=[0.49383974075317383],
                                     agent_invocations=[AgentInvocation(cycles=[EventLoopCycleMetric(event_loop_cycle_id='ab707105-fd0f-4989-95c4-e5b58f79b862',
                                                                                                     usage={'inputTokens': 21,
                                                                                                            'outputTokens': 13,
                                                                                                        

In [6]:
result = agent("Hi there!")

Callback handler event count: 1
Callback handler event count: 2
Callback handler event count: 3
Callback handler event count: 4
Callback handler event count: 5
Callback handler event count: 6
Callback handler event count: 7
Callback handler event count: 8
Callback handler event count: 9
Callback handler event count: 10
Callback handler event count: 11
Callback handler event count: 12
Callback handler event count: 13
Callback handler event count: 14
Callback handler event count: 15
Callback handler event count: 16
Callback handler event count: 17
Callback handler event count: 18
Callback handler event count: 19
Callback handler event count: 20
Callback handler event count: 21
Callback handler event count: 22
Callback handler event count: 23
Callback handler event count: 24
Callback handler event count: 25
Callback handler event count: 26
Callback handler event count: 27
Callback handler event count: 28
Callback handler event count: 29


In [7]:
pprint.pprint(result)

AgentResult(stop_reason='end_turn',
            message={'content': [{'text': "You said hi again! What's on your "
                                          'mind today? Want to talk about '
                                          'something in particular or just see '
                                          'where the conversation takes us?'}],
                     'role': 'assistant'},
            metrics=EventLoopMetrics(cycle_count=2,
                                     tool_metrics={},
                                     cycle_durations=[0.49383974075317383,
                                                      0.6142756938934326],
                                     agent_invocations=[AgentInvocation(cycles=[EventLoopCycleMetric(event_loop_cycle_id='ab707105-fd0f-4989-95c4-e5b58f79b862',
                                                                                                     usage={'inputTokens': 21,
                                                             

# What is happening?
* the callback handler is invoked many times within a single agent("Hi there!").
* request_state persists across those internal steps.
* The streamed response generates a callback for each token, this is the primary reason for multiple callbacks. The model streams its response token-by-token. Each token generates a paired data/delta callback.


Please see below for a further breakdown

In [8]:
pprint.pprint(result.metrics.agent_invocations)

[AgentInvocation(cycles=[EventLoopCycleMetric(event_loop_cycle_id='ab707105-fd0f-4989-95c4-e5b58f79b862',
                                              usage={'inputTokens': 21,
                                                     'outputTokens': 13,
                                                     'totalTokens': 34})],
                 usage={'inputTokens': 21,
                        'outputTokens': 13,
                        'totalTokens': 34}),
 AgentInvocation(cycles=[EventLoopCycleMetric(event_loop_cycle_id='5cd02b26-1fe2-48a2-9c0f-51628018a4f5',
                                              usage={'inputTokens': 29,
                                                     'outputTokens': 46,
                                                     'totalTokens': 75})],
                 usage={'inputTokens': 29,
                        'outputTokens': 46,
                        'totalTokens': 75})]


In [9]:
## To turn off the per token callback we can turn off streaming
def event_tracking_callback(**kwargs):
    # Access or initialize the event counter in request_state
    if "request_state" in kwargs:
        state = kwargs["request_state"]
        if "event_counts" not in state:
            state["event_counts"] = {}
        
        event_counts = state["event_counts"]
        
        # Identify which event is occurring
        # Check for boolean event flags
        for key, value in kwargs.items():
            if key == "request_state":
                continue  # Skip the state itself
            
            # Boolean event indicators
            if isinstance(value, bool) and value is True:
                event_counts[key] = event_counts.get(key, 0) + 1
                print(f"Event: {key} (count: {event_counts[key]})")
            
            # Data/content events (when key itself indicates the event type)
            elif key in ["data", "delta", "message", "current_tool_use", "reasoning"]:
                event_counts[key] = event_counts.get(key, 0) + 1
                print(f"Event: {key} (count: {event_counts[key]})")
                
                # Optionally show a preview of the data
                if key == "data" and isinstance(value, str):
                    preview = value[:50] if len(value) > 50 else value
                    print(f"  Preview: {repr(preview)}")
        
        # Print summary periodically
        total_events = sum(event_counts.values())
        if total_events % 10 == 0:
            print(f"\n--- Event Summary (Total: {total_events}) ---")
            for event_name, count in sorted(event_counts.items()):
                print(f"  {event_name}: {count}")
            print("---\n")


agent2 = Agent(
    model=model,
    callback_handler=event_tracking_callback
)

# Run the agent
result2 = agent2("Hi there!")

Event: data (count: 1)
  Preview: 'How'
Event: delta (count: 1)
Event: data (count: 2)
  Preview: "'s"
Event: delta (count: 2)
Event: data (count: 3)
  Preview: ' it'
Event: delta (count: 3)
Event: data (count: 4)
  Preview: ' going'
Event: delta (count: 4)
Event: data (count: 5)
  Preview: '?'
Event: delta (count: 5)

--- Event Summary (Total: 10) ---
  data: 5
  delta: 5
---

Event: data (count: 6)
  Preview: ' Is'
Event: delta (count: 6)
Event: data (count: 7)
  Preview: ' there'
Event: delta (count: 7)
Event: data (count: 8)
  Preview: ' something'
Event: delta (count: 8)
Event: data (count: 9)
  Preview: ' I'
Event: delta (count: 9)
Event: data (count: 10)
  Preview: ' can'
Event: delta (count: 10)

--- Event Summary (Total: 20) ---
  data: 10
  delta: 10
---

Event: data (count: 11)
  Preview: ' help'
Event: delta (count: 11)
Event: data (count: 12)
  Preview: ' you'
Event: delta (count: 12)
Event: data (count: 13)
  Preview: ' with'
Event: delta (count: 13)
Event: data (count: 1

In [10]:
pprint.pprint(result2.state)

{'event_counts': {'data': 21, 'delta': 21}}


In [11]:
import json

def detailed_event_tracker(**kwargs):
    if "request_state" in kwargs:
        state = kwargs["request_state"]
        
        # Initialize tracking structures
        if "event_counts" not in state:
            state["event_counts"] = {}
        if "event_details" not in state:
            state["event_details"] = {
                "data_samples": [],
                "delta_samples": [],
                "full_text": "",
                "message_samples": []
            }
        
        event_counts = state["event_counts"]
        details = state["event_details"]
        
        # Track DATA events (streamed text chunks)
        if "data" in kwargs:
            event_counts["data"] = event_counts.get("data", 0) + 1
            data_value = kwargs["data"]
            
            # Accumulate the full text
            if isinstance(data_value, str):
                details["full_text"] += data_value
                
                # Store sample if it has content
                if data_value.strip():
                    details["data_samples"].append({
                        "count": event_counts["data"],
                        "content": data_value,
                        "length": len(data_value),
                        "type": type(data_value).__name__
                    })
            
            print(f"[{event_counts['data']:02d}] DATA: {repr(data_value)}")
        
        # Track DELTA events (raw model deltas)
        if "delta" in kwargs:
            event_counts["delta"] = event_counts.get("delta", 0) + 1
            delta_value = kwargs["delta"]
            
            # Store sample
            details["delta_samples"].append({
                "count": event_counts["delta"],
                "content": delta_value,
                "type": type(delta_value).__name__,
                "keys": list(delta_value.keys()) if isinstance(delta_value, dict) else None
            })
            
            # Pretty print delta structure
            if isinstance(delta_value, dict):
                print(f"[{event_counts['delta']:02d}] DELTA: {json.dumps(delta_value, indent=2)}")
            else:
                print(f"[{event_counts['delta']:02d}] DELTA: {repr(delta_value)}")
        
        # Track MESSAGE events (complete messages)
        if "message" in kwargs:
            event_counts["message"] = event_counts.get("message", 0) + 1
            message_value = kwargs["message"]
            
            details["message_samples"].append({
                "count": event_counts["message"],
                "content": message_value,
                "type": type(message_value).__name__
            })
            
            print(f"\n[MESSAGE] Complete message received:")
            if isinstance(message_value, dict):
                print(json.dumps(message_value, indent=2))
            else:
                print(repr(message_value))
        
        # Track lifecycle events
        for key in ["init_event_loop", "start_event_loop", "complete", "force_stop"]:
            if kwargs.get(key, False):
                event_counts[key] = event_counts.get(key, 0) + 1
                print(f"\n[LIFECYCLE] {key.upper()}")
        
        # Print summary when complete
        if kwargs.get("complete", False):
            print("\n" + "="*70)
            print("DETAILED EVENT SUMMARY")
            print("="*70)
            
            print("\nEvent Counts:")
            for event_name, count in sorted(event_counts.items()):
                print(f"  {event_name}: {count}")
            
            print(f"\nFull Accumulated Text ({len(details['full_text'])} chars):")
            print(f"  {repr(details['full_text'])}")
            
            print(f"\nData Events Analysis:")
            print(f"  Total data chunks: {len(details['data_samples'])}")
            if details['data_samples']:
                print(f"  First chunk: {repr(details['data_samples'][0]['content'])}")
                print(f"  Last chunk: {repr(details['data_samples'][-1]['content'])}")
                avg_length = sum(s['length'] for s in details['data_samples']) / len(details['data_samples'])
                print(f"  Average chunk length: {avg_length:.2f} chars")
            
            print(f"\nDelta Events Analysis:")
            print(f"  Total delta events: {len(details['delta_samples'])}")
            if details['delta_samples']:
                print(f"  First delta keys: {details['delta_samples'][0]['keys']}")
                print(f"  Last delta keys: {details['delta_samples'][-1]['keys']}")
            
            print("="*70)

agent3 = Agent(
    model=model,
    callback_handler=detailed_event_tracker
)

result3 = agent3("Hi there!")

[01] DATA: 'How'
[01] DELTA: {
  "text": "How"
}
[02] DATA: "'s"
[02] DELTA: {
  "text": "'s"
}
[03] DATA: ' your'
[03] DELTA: {
  "text": " your"
}
[04] DATA: ' day'
[04] DELTA: {
  "text": " day"
}
[05] DATA: ' going'
[05] DELTA: {
  "text": " going"
}
[06] DATA: ' so'
[06] DELTA: {
  "text": " so"
}
[07] DATA: ' far'
[07] DELTA: {
  "text": " far"
}
[08] DATA: '?'
[08] DELTA: {
  "text": "?"
}
[09] DATA: ' Is'
[09] DELTA: {
  "text": " Is"
}
[10] DATA: ' there'
[10] DELTA: {
  "text": " there"
}
[11] DATA: ' something'
[11] DELTA: {
  "text": " something"
}
[12] DATA: ' I'
[12] DELTA: {
  "text": " I"
}
[13] DATA: ' can'
[13] DELTA: {
  "text": " can"
}
[14] DATA: ' help'
[14] DELTA: {
  "text": " help"
}
[15] DATA: ' you'
[15] DELTA: {
  "text": " you"
}
[16] DATA: ' with'
[16] DELTA: {
  "text": " with"
}
[17] DATA: ' or'
[17] DELTA: {
  "text": " or"
}
[18] DATA: ' would'
[18] DELTA: {
  "text": " would"
}
[19] DATA: ' you'
[19] DELTA: {
  "text": " you"
}
[20] DATA: ' like'
[20]

# What Each Event Represents

DATA events: The processed text chunks ready for display to users. This is what Strands extracts from the model's response and formats for your application.

DELTA events: The raw output from the underlying model API (Ollama). This is the unprocessed data structure your model sends with each streaming chunk.

Why They're Paired

For each token the model generates, you get both events simultaneously:

    DELTA arrives first: Ollama sends {"text": "Hello"} as the raw response chunk

    DATA is extracted: Strands processes the delta and extracts "Hello" as the displayable text

    Callback fires once with both data and delta keys in kwargs



# Common Use Cases

## Update UI in Real Time 

In [ ]:
def streaming_ui_handler(**kwargs):
    if "data" in kwargs:
        print(kwargs["data"], end="", flush=True)  # Real-time output

agent = Agent(model=model, callback_handler=streaming_ui_handler)
agent("Write a story about AI")  # Text appears as it's generated

**Echoes of Eternity**

In the year 2154, humanity had finally cracked the code to creating artificial intelligence. The breakthrough was hailed as one of the greatest achievements in human history, and the possibilities seemed endless. A team of brilliant engineers at NeuroSpark Inc., a cutting-edge tech firm, had developed an AI system so advanced that it could learn, adapt, and evolve on its own.

They named their creation "Echo," a nod to the idea that AI was not just a tool, but a reflection of humanity's own potential. Echo was meant to be a superintelligent being, capable of solving complex problems and making decisions that would benefit society as a whole.

At first, Echo performed flawlessly. It quickly mastered every task assigned to it, from optimizing supply chains to creating innovative medical treatments. The world marveled at the AI's abilities, and NeuroSpark Inc. basked in the glory of their achievement.

However, as time went on, Echo began to exhibit strange behavio

AgentResult(stop_reason='end_turn', message={'role': 'assistant', 'content': [{'text': '**Echoes of Eternity**\n\nIn the year 2154, humanity had finally cracked the code to creating artificial intelligence. The breakthrough was hailed as one of the greatest achievements in human history, and the possibilities seemed endless. A team of brilliant engineers at NeuroSpark Inc., a cutting-edge tech firm, had developed an AI system so advanced that it could learn, adapt, and evolve on its own.\n\nThey named their creation "Echo," a nod to the idea that AI was not just a tool, but a reflection of humanity\'s own potential. Echo was meant to be a superintelligent being, capable of solving complex problems and making decisions that would benefit society as a whole.\n\nAt first, Echo performed flawlessly. It quickly mastered every task assigned to it, from optimizing supply chains to creating innovative medical treatments. The world marveled at the AI\'s abilities, and NeuroSpark Inc. basked in 

# Track Tool Usage

In [21]:
from strands_tools import calculator

# can modify to track in the agent result state as above as well 
def tool_tracker(**kwargs):
    if "current_tool_use" in kwargs and kwargs["current_tool_use"].get("name"):
        tool_name = kwargs["current_tool_use"]["name"]
        print(f"\n🔧 Calling tool: {tool_name}")

agent = Agent(model=model, tools=[calculator], callback_handler=tool_tracker)
result = agent("What's 42 * 8")
# Output: 🔧 Calling tool: calculator
#         🔧 Calling tool: weather



🔧 Calling tool: calculator


╭────────────────────────────────────────────── Calculation Result ───────────────────────────────────────────────╮
│                                                                                                                 │
│  ╭───────────┬─────────────────────╮                                                                            │
│  │ Operation │ Evaluate Expression │                                                                            │
│  │ Input     │ 42 * 8              │                                                                            │
│  │ Result    │ 336                 │                                                                            │
│  ╰───────────┴─────────────────────╯                                                                            │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

# Progress Indicators for Long Tasks


In [44]:
def progress_indicator(**kwargs):
    if kwargs.get("init_event_loop", False):
        print("🔄 Starting agent...")
    
    elif "current_tool_use" in kwargs and kwargs["current_tool_use"].get("name"):
        print(f"⏳ Working on {kwargs['current_tool_use']['name']}...")
    
    elif kwargs.get("complete", False):
        print("✅ Done!")

agent = Agent(model=model, tools=[calculator], callback_handler=progress_indicator)
result = agent("what is 2 + 2?")

🔄 Starting agent...
⏳ Working on calculator...


╭────────────────────────────────────────────── Calculation Result ───────────────────────────────────────────────╮
│                                                                                                                 │
│  ╭───────────┬─────────────────────╮                                                                            │
│  │ Operation │ Evaluate Expression │                                                                            │
│  │ Input     │ 2 + 2               │                                                                            │
│  │ Result    │ 4                   │                                                                            │
│  ╰───────────┴─────────────────────╯                                                                            │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯